In [3]:
filepath="C:\\Users\\asus\\600153.csv"
import pandas as pd
import matplotlib
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'
from pyecharts.charts import Bar, Pie, Geo, Liquid
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ChartType, SymbolType

df = pd.read_csv(filepath,encoding='gb18030')
print(df)
df=df.iloc[::-1].reset_index(drop= False)
print(df)
df = df.dropna(subset=['收盘价'])
historydata=df[["日期","收盘价","送股","分红"]]
class test:
    """test"""
    steps = 0#move o 0,havemove 为-1
    price = 0
    tax=0
    trade_date=0
    fenhong=0
    songgu=0
    predictprice = 0
    historydata = 0
    nowamplitude = 0
    allamplitude = []
    cashmoney = 1660.0
    stock = 0
    transactionrecord = [] 
    def __init__(self,historydata):
        self.historydata = historydata 
    def dopredictprice(self):
        """预测当天的回归均值价格"""
        y_arr = self.historydata["收盘价"][0:self.steps].values.tolist()
        x_arr = self.historydata["收盘价"][0:self.steps].index.tolist()
        #print(y_arr) 
        #print(x_arr)
        x_b_arr = sm.add_constant(x_arr)  # 添加常数列1
        model = regression.linear_model.OLS(y_arr, x_b_arr).fit()  # 使用OLS做拟合
        rad = model.params[1]  # y = kx + b :params[1] = k
        intercept = model.params[0]  # y = kx + b :params[0] = b
        self.predictprice = round((self.steps-1) * rad + intercept,2)
    def move(self):
        """交易日进入到下一天"""
        self.steps = self.steps+1
        self.trade_date = self.historydata["日期"][self.steps-1]
        self.price = self.historydata["收盘价"][self.steps-1]
        self.fenhong = self.historydata["分红"][self.steps-1]
        self.songgu = self.historydata["送股"][self.steps-1]
        self.dopredictprice()
        self.nowamplitude = round((self.price-self.predictprice)/self.predictprice*100,2)
        self.allamplitude.append(round(self.nowamplitude,2))
        if self.stock>0  and self.fenhong!=0:
            self.cashmoney=self.cashmoney+self.stock*self.fenhong
            self.stock=self.stock+self.stock*self.songgu
            print("当前持股",self.stock,"分红日期",self.trade_date,"现金",self.cashmoney,"分红金额",self.fenhong*self.stock)
            
        if self.nowamplitude<-15 and self.price!=0:
            self.buy()
        elif self.stock>0 and self.nowamplitude>15 and self.price!=0and self.price*100>self.transactionrecord[len(self.transactionrecord)-1]["买入金额"]:
            self.sale()
    def handmove(self,steps):
        """交易日进入到下一天"""
        self.steps=self.steps+1
        for i in range(self.steps,self.steps+steps):
            self.steps = i
            self.trade_date = self.historydata["日期"][self.steps]
            self.price = self.historydata["收盘价"][self.steps]
            self.fenhong = self.historydata["分红"][self.steps]
            self.songgu = self.historydata["送股"][self.steps]
            if self.stock>0  and self.fenhong!=0:#移动的过程中，如果有股价或者分红，把股价或者分红给买方。
                self.cashmoney=round(self.cashmoney,2)+round(self.stock*self.fenhong,2)
                print("分红",self.fenhong*self.stock)
                self.stock=self.stock+self.stock*self.songgu
                #print(self.stock,self.trade_date,self.cashmoney,"---")
        print("移动到交易日期",self.stock,self.trade_date,self.cashmoney,"---")
                
           

    def buy(self):
        """买入股票"""
        if int(self.cashmoney/self.price/100)*100>0 :
            stock =int(self.cashmoney/self.price/100)*100
            self.tax = max(round(self.price*stock*0.0003+0.02,2),round(5+0.02,2)) if stock!=0 else 0
            self.cashmoney = round(self.cashmoney-round(stock*self.price,2)-self.tax,2)
            self.stock=self.stock+stock
            transactionrecord={}
            transactionrecord['日期'] = self.trade_date
            transactionrecord['类型'] = "买入" 
            transactionrecord['股票数量'] = self.stock 
            transactionrecord['股票单价'] = self.price 
            transactionrecord['买入金额'] = self.price* self.stock 
            transactionrecord['税费'] = self.tax 
            transactionrecord['账户余额'] = self.cashmoney 
            self.transactionrecord.append(transactionrecord)
            print("买入股票",self.transactionrecord[len(self.transactionrecord)-1])
        else:
            print("现金不足买一百股")
      

    def sale(self):
        """卖出股票"""
        if self.stock>0:
            self.tax = max(round(self.price*self.stock*0.0003+0.02,2),round(5+0.02,2))+round(self.price*self.stock*0.001,2)
            self.cashmoney = round(self.cashmoney+self.stock*self.price-self.tax,2)
            transactionrecord={}
            transactionrecord['日期'] = self.trade_date
            transactionrecord['类型'] = "卖出" 
            transactionrecord['股票数量'] = self.stock 
            transactionrecord['股票单价'] = self.price 
            transactionrecord['卖出金额'] = self.price* self.stock 
            transactionrecord['税费'] = self.tax 
            transactionrecord['账户余额'] = self.cashmoney
            self.transactionrecord.append(transactionrecord)
            self.stock =0
            print("卖出股票")
        else:
            print("没有股票可以卖")
    def clear(self):
        """清空重置"""
        self.steps = 0#move o 0,havemove 为-1
        self.price = 0
        self.tax=0
        self.trade_date=0
        self.fenhong=0
        self.songgu=0
        self.predictprice = 0
        self.nowamplitude = 0
        self.allamplitude = []
        self.cashmoney = 166000.0
        self.stock = 0
        self.transactionrecord = [] 
test=test(historydata)


for i in range(1,5391):
    test.move()

             日期     股票代码    名称    收盘价    最高价    最低价    开盘价    前收盘    涨跌额  \
0      2020-9-4  '600153  建发股份   9.40   9.56   9.32   9.35   9.31   0.09   
1      2020-9-3  '600153  建发股份   9.31   9.45   9.29   9.43   9.35  -0.04   
2      2020-9-2  '600153  建发股份   9.35   9.43   9.27   9.38   9.38  -0.03   
3      2020-9-1  '600153  建发股份   9.38   9.53   9.32   9.51   9.62  -0.24   
4     2020-8-31  '600153  建发股份   9.62   9.93   9.51   9.88   9.83  -0.21   
...         ...      ...   ...    ...    ...    ...    ...    ...    ...   
5386  1998-6-22  '600153  厦门建发  17.29  17.49  16.52  16.93  16.83   0.46   
5387  1998-6-19  '600153  厦门建发  16.83  17.11  15.97  15.97  15.95   0.88   
5388  1998-6-18  '600153  厦门建发  15.95  16.50  15.88  16.20  16.16  -0.21   
5389  1998-6-17  '600153  厦门建发  16.16  16.16  14.83  15.20  14.69   1.47   
5390  1998-6-16  '600153  厦门建发  14.69  15.00  14.00  15.00   0.00   None   

          涨跌幅      换手率       成交量       成交金额          总市值         流通市值   送股  \
0      0.

现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现金不足买一百股
现

In [ ]:
test.transactionrecord

In [4]:
pd.DataFrame(test.allamplitude).to_excel('分析.xls')

In [206]:
#股票回测方法2 ，根据单价低于市值成交金额百分之八十买入或者高于百分之八十卖出
filepath="C:\\Users\\asus\\600153.csv"
import pandas as pd
import matplotlib
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'
from pyecharts.charts import Bar, Pie, Geo, Liquid
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ChartType, SymbolType

df = pd.read_csv(filepath,encoding='gb18030')
print(df.columns)
df=df.iloc[::-1].reset_index(drop= True)
df = df.dropna(subset=['收盘价'])
historydata=df[["日期","收盘价","送股","分红",'成交量', '成交金额', '总市值']]
print(historydata.head(10))
class test:
    """test"""
    steps = 0#move o 0,havemove 为-1
    price = 0
    tax=0
    trade_date=0
    fenhong=0
    songgu=0
    chengjiaoliang=0
    chengjiaojine=0
    zongshizhi=0
    predictprice = 0
    historydata = 0
    nowamplitude = 0
    allamplitude = []
    cashmoney = 100000.0
    stock = 0
    transactionrecord = [] 
    def __init__(self,historydata):
        self.historydata = historydata 
    def dopredictprice(self):
        """预测当天的回归均值价格"""
        y_arr = self.historydata["收盘价"][0:self.steps].values.tolist()
        x_arr = self.historydata["收盘价"][0:self.steps].index.tolist()
        #print(y_arr) 
        #print(x_arr)
        x_b_arr = sm.add_constant(x_arr)  # 添加常数列1
        model = regression.linear_model.OLS(y_arr, x_b_arr).fit()  # 使用OLS做拟合
        rad = model.params[1]  # y = kx + b :params[1] = k
        intercept = model.params[0]  # y = kx + b :params[0] = b
        self.predictprice = round((self.steps-1) * rad + intercept,2)
    def minmax(self,num):
        num=num
        cumsumchengjiaojine=0
        n=0
        l=[]
        result=[]
        while cumsumchengjiaojine<self.historydata["总市值"][num-1]:
            n=n+1
            cumsumchengjiaojine=cumsumchengjiaojine+self.historydata["成交金额"][num-n]
            l.append(int(num-n))
        pd1=self.historydata.iloc[l,:].sort_values(by=['收盘价'],ascending = True)[["成交量"]].cumsum()
        pd1.columns=["开累成交量"]
        pd2=self.historydata.iloc[l,:].sort_values(by=['收盘价'],ascending = True)
        pd3=pd.concat([pd1,pd2],axis=1)
        result.append(pd3[(pd3.开累成交量>pd3.tail(1)["开累成交量"].values[0]*0.2)].head(1)["收盘价"].values[0])
        result.append(pd3[(pd3.开累成交量>pd3.tail(1)["开累成交量"].values[0]*0.8)].head(1)["收盘价"].values[0])
        return result       
        
    def move(self):
        """交易日进入到下一天"""
        self.steps = self.steps+1
        self.trade_date = self.historydata["日期"][self.steps-1]
        self.price = self.historydata["收盘价"][self.steps-1]
        self.fenhong = self.historydata["分红"][self.steps-1]
        self.songgu = self.historydata["送股"][self.steps-1]
        self.chengjiaoliang=self.historydata["成交量"][self.steps-1]
        self.chengjiaojine=self.historydata["成交金额"][self.steps-1]
        self.zongshizhi=self.historydata["总市值"][self.steps-1]
        if self.stock>0  and self.fenhong!=0:
            self.cashmoney=self.cashmoney+self.stock*self.fenhong
            self.stock=self.stock+self.stock*self.songgu
            print("当前持股",self.stock,"分红日期",self.trade_date,"现金",self.cashmoney,"分红金额",self.fenhong*self.stock)
        
        if self.price<self.minmax(self.steps)[0]and self.price>0 :
            self.buy()
            print(self.minmax(self.steps)[0],self.minmax(self.steps)[1])
        elif self.stock>0 and self.price>self.minmax(self.steps)[1]:
            self.sale()
    def handmove(self,steps):
        """交易日进入到下一天"""
        self.steps=self.steps+1
        for i in range(self.steps,self.steps+steps):
            self.steps = i
            self.trade_date = self.historydata["日期"][self.steps]
            self.price = self.historydata["收盘价"][self.steps]
            self.fenhong = self.historydata["分红"][self.steps]
            self.songgu = self.historydata["送股"][self.steps]
            if self.stock>0  and self.fenhong!=0:#移动的过程中，如果有股价或者分红，把股价或者分红给买方。
                self.cashmoney=round(self.cashmoney,2)+round(self.stock*self.fenhong,2)
                print("分红",self.fenhong*self.stock)
                self.stock=self.stock+self.stock*self.songgu
                #print(self.stock,self.trade_date,self.cashmoney,"---")
        print("移动到交易日期",self.stock,self.trade_date,self.cashmoney,"---")
                
           

    def buy(self):
        """买入股票"""
        if int(self.cashmoney/self.price/100)*100>0 :
            stock =int(self.cashmoney/self.price/100)*100
            self.tax = max(round(self.price*stock*0.0003+0.02,2),round(5+0.02,2)) if stock!=0 else 0
            self.cashmoney = round(self.cashmoney-round(stock*self.price,2)-self.tax,2)
            self.stock=self.stock+stock
            transactionrecord={}
            transactionrecord['日期'] = self.trade_date
            transactionrecord['类型'] = "买入" 
            transactionrecord['股票数量'] = self.stock 
            transactionrecord['股票单价'] = self.price 
            transactionrecord['买入金额'] = self.price* self.stock 
            transactionrecord['税费'] = self.tax 
            transactionrecord['账户余额'] = self.cashmoney 
            self.transactionrecord.append(transactionrecord)
            print("买入股票",self.transactionrecord[len(self.transactionrecord)-1])
        else:
            print("现金不足买一百股")
      

    def sale(self):
        """卖出股票"""
        if self.stock>0:
            self.tax = max(round(self.price*self.stock*0.0003+0.02,2),round(5+0.02,2))+round(self.price*self.stock*0.001,2)
            self.cashmoney = round(self.cashmoney+self.stock*self.price-self.tax,2)
            transactionrecord={}
            transactionrecord['日期'] = self.trade_date
            transactionrecord['类型'] = "卖出" 
            transactionrecord['股票数量'] = self.stock 
            transactionrecord['股票单价'] = self.price 
            transactionrecord['卖出金额'] = self.price* self.stock 
            transactionrecord['税费'] = self.tax 
            transactionrecord['账户余额'] = self.cashmoney
            self.transactionrecord.append(transactionrecord)
            self.stock =0
            print("卖出股票")
        else:
            print("没有股票可以卖")
    def clear(self):
        """清空重置"""
        self.steps = 0#move o 0,havemove 为-1
        self.price = 0
        self.tax=0
        self.trade_date=0
        self.fenhong=0
        self.songgu=0
        self.predictprice = 0
        self.nowamplitude = 0
        self.allamplitude = []
        self.cashmoney = 166000.0
        self.stock = 0
        self.transactionrecord = [] 
test=test(historydata)



Index(['日期', '股票代码', '名称', '收盘价', '最高价', '最低价', '开盘价', '前收盘', '涨跌额', '涨跌幅',
       '换手率', '成交量', '成交金额', '总市值', '流通市值', '送股', '分红'],
      dtype='object')
          日期    收盘价   送股   分红       成交量       成交金额         总市值
0  1998-6-16  14.69  0.0  0.0  26800332  385631050  2717650000
1  1998-6-17  16.16  0.0  0.0  14293506  221556989  2989600000
2  1998-6-18  15.95  0.0  0.0  10175834  164864634  2950750000
3  1998-6-19  16.83  0.0  0.0   5616245   93496166  3113550000
4  1998-6-22  17.29  0.0  0.0   5053482   86081669  3198650000
5  1998-6-23  16.83  0.0  0.0   3230365   54728032  3113550000
6  1998-6-24  16.85  0.0  0.0   2314050   39210498  3117250000
7  1998-6-25  17.53  0.0  0.0   3038208   52653712  3243050000
8  1998-6-26  17.33  0.0  0.0   1688735   29405755  3206050000
9  1998-6-29  16.50  0.0  0.0   2216354   36829135  3052500000


In [207]:
test.handmove(500)
for i in range(500,5391):
    test.move()

移动到交易日期 0 2000-7-11 100000.0 ---
买入股票 {'日期': '2000-10-12', '类型': '买入', '股票数量': 5400, '股票单价': 18.5, '买入金额': 99900.0, '税费': 29.99, '账户余额': 70.01}
18.73 25.0
现金不足买一百股
18.61 25.0
现金不足买一百股
18.6 25.0
现金不足买一百股
18.6 25.0
现金不足买一百股
18.6 25.0
现金不足买一百股
18.58 25.0
现金不足买一百股
18.6 25.0
现金不足买一百股
18.6 25.0
现金不足买一百股
18.58 25.0
现金不足买一百股
18.51 25.0
现金不足买一百股
18.51 25.0
现金不足买一百股
18.51 25.0
现金不足买一百股
18.5 25.0
现金不足买一百股
18.48 25.0
现金不足买一百股
18.48 25.0
现金不足买一百股
18.47 25.0
现金不足买一百股
18.48 25.0
现金不足买一百股
18.5 25.0
现金不足买一百股
18.51 25.0
现金不足买一百股
18.7 24.62
现金不足买一百股
18.7 24.62
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.93 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.99 25.0
现金不足买一百股
18.99 25.0
现金不足买一百股
18.99 25.0
现金不足买一百股
18.9 25.0
现金不足买一百股
18.88 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.9 25.0
现金不足买一百股
18.9 25.0
现金不足买一百股
18.88 25.0
现金不足买一百股
18.93 25.0
现金不足买一百股
18.92 25.0
现金不足买一百股
18.88 25.0
现金不足买一百股
18.88 25.0
现金不足买一百股
18.88 

12.41 17.48
现金不足买一百股
12.41 17.48
现金不足买一百股
12.4 17.2
现金不足买一百股
12.39 15.95
现金不足买一百股
12.38 16.16
现金不足买一百股
12.38 17.2
现金不足买一百股
12.38 17.2
现金不足买一百股
12.39 17.48
现金不足买一百股
12.38 17.48
现金不足买一百股
12.35 17.48
现金不足买一百股
12.35 17.46
现金不足买一百股
12.35 17.55
现金不足买一百股
12.32 17.48
现金不足买一百股
12.32 17.55
现金不足买一百股
12.31 17.48
现金不足买一百股
12.27 17.45
现金不足买一百股
12.23 16.16
现金不足买一百股
12.23 15.95
现金不足买一百股
12.21 15.9
现金不足买一百股
12.16 15.43
现金不足买一百股
12.14 15.43
现金不足买一百股
12.15 15.43
现金不足买一百股
12.15 15.43
现金不足买一百股
12.16 15.63
现金不足买一百股
12.17 15.9
现金不足买一百股
12.15 15.86
现金不足买一百股
12.15 15.9
现金不足买一百股
12.15 16.18
现金不足买一百股
12.13 15.95
现金不足买一百股
12.13 17.2
现金不足买一百股
12.13 17.2
现金不足买一百股
12.11 16.18
现金不足买一百股
12.02 15.95
现金不足买一百股
12.02 15.95
现金不足买一百股
12.0 16.18
现金不足买一百股
12.0 16.18
现金不足买一百股
11.95 16.16
现金不足买一百股
11.93 15.95
现金不足买一百股
11.91 15.95
现金不足买一百股
11.89 15.63
现金不足买一百股
11.89 15.79
现金不足买一百股
11.89 15.95
现金不足买一百股
11.88 15.86
现金不足买一百股
11.88 15.9
现金不足买一百股
11.86 15.79
现金不足买一百股
11.86 15.95
现金不足买一百股
11.82 15.95
现金不足买一百股
11.8 15.95
现金不足买一百股
11.76

现金不足买一百股
5.39 6.22
现金不足买一百股
5.39 6.19
现金不足买一百股
5.38 6.19
现金不足买一百股
5.38 6.13
现金不足买一百股
5.38 6.19
现金不足买一百股
5.38 6.13
现金不足买一百股
5.38 6.13
现金不足买一百股
5.38 6.13
现金不足买一百股
5.38 6.13
现金不足买一百股
5.38 5.99
现金不足买一百股
5.37 5.9
现金不足买一百股
5.36 5.84
现金不足买一百股
5.36 5.81
现金不足买一百股
5.36 5.81
现金不足买一百股
5.36 5.81
现金不足买一百股
5.36 5.78
现金不足买一百股
5.35 5.78
现金不足买一百股
5.35 5.77
现金不足买一百股
5.34 5.77
现金不足买一百股
5.34 5.77
现金不足买一百股
5.34 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.35 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.33 5.77
现金不足买一百股
5.34 5.77
现金不足买一百股
5.32 5.77
现金不足买一百股
5.31 5.77
现金不足买一百股
5.31 5.77
现金不足买一百股
5.31 5.77
现金不足买一百股
5.31 5.76
现金不足买一百股
5.3 5.76
现金不足买一百股
5.3 5.77
现金不足买一百股
5.3 5.77
现金不足买一百股
5.29 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.28 5.75
现金不足买一百股
5.26 5.74
现金不足买一百股
5.26 5.75
现金不足买一百股
5.26 5.75
现金不足买一百股
5.25 5.74
现金不足买一百股
5.25 5.74
现金不足买一百股
5.23 5.74
现金不足买一百股
5.22 5.72
现金不足买一百股
5.21 5.

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(test.sum()).to_excel('分析.xls')

In [3]:
import tkinter
top = tkinter.Tk()
# 进入消息循环
top.mainloop()